In [192]:
import os
import pandas as pd

# get all files starting with "xval_" in results
#files = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and "xval_2_5" in f or "xval_3_5" in f and "old" not in f]
files = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_fad_uniform_many_lengthscale_22_5" in f)]
# for files, xp_name in zip([files_6_op, files_4_op, files_4_op_mlm, files_4_op_small_model, files_4_op_uniform, files_unknown, files_varying_lengthscale, files_different_depth],
#                            ["6_op", "4_op", "4_op_mlm", "4_op_small_model", "4_op_uniform", "unknown", "varying_lengthscale", "different_depth"]):
#     for file in files:
#         df = pd.read_csv(f"results/{file}")#.sample(10_000)
#         df["experiment"] = xp_name
#         results.append(df)
results = []
for file in files:
    df = pd.read_csv(f"results/{file}")
    df["experiment"] = "fad_uniform_many_lengthscale_22_5"
    results.append(df)

# concatenate the dataframes
results = pd.concat(results)
#results = df

# remove missing columns
results = results.dropna(axis=1, how="all")
results["epoch"].unique()
results[results["experiment"] == "unknown"]["epoch"].unique()
results = results.fillna("missing")
results = results.applymap(str)
results["loss_mlm"] = results["loss_mlm"].astype(float)
results["loss_num"] = results["loss_num"].astype(float)
results["loss_total"] = results["loss_total"].astype(float)
results["val_loss_mlm"] = results["val_loss_mlm"].astype(float)
results["val_loss_num"] = results["val_loss_num"].astype(float)
results["val_loss_total"] = results["val_loss_total"].astype(float)
results["epoch"] = results["epoch"].astype(int)
results.columns

results["task"] = "arithmetic"

# restrict to epoch = 1
results = results[results["epoch"] == 1]

/var/folders/fy/__8z8cpn6gs04465sq9g1nq80000gn/T/ipykernel_35860/2863908263.py:28: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [193]:
df = pd.read_csv("results_summarised/sort_different_lengthscale_22_05.csv")

# convert test_loss_list into two columns epoch and test_loss
import ast
import pandas as pd

# Assuming df is your DataFrame and 'test_loss_list' is the column you're working with.

for index, row in df.iterrows():
    try:
        # Try to evaluate the row as a literal.
        pd.eval(row['test_loss_list'])
    except Exception as e:
        # If an exception occurs, print the index and the value that failed
        print(f"Failed at index {index}: {row['test_loss_list']}")
        print(f"Error: {e}")
        # Optionally, break after the first failure
        #break
import ast
# replace "nan" with None
df['test_loss_list'] = df['test_loss_list'].str.replace('nan', 'None')
df['loss_list'] = df['loss_list'].str.replace('nan', 'None')
#df['test_loss_list'] = df['test_loss_list'].apply(lambda s: pd.eval(s))
#df['loss_list'] = df['loss_list'].apply(lambda s: pd.eval(s))

df['test_loss_list'] = df['test_loss_list'].apply(ast.literal_eval)
df['loss_list'] = df['loss_list'].apply(ast.literal_eval)

# Create a new DataFrame for Plotly Express
rows_list = []
for _, row in df.iterrows():
    params = row.to_dict()
    test_loss_list = params.pop('test_loss_list')
    loss_list = params.pop('loss_list')
    for epoch, loss in enumerate(test_loss_list):
        params['epoch'] = epoch
        params['test_loss'] = loss
        params['train_loss'] = loss_list[epoch]
        rows_list.append(params.copy())  # Use .copy() to avoid modifying the original params

# Create a new DataFrame
df_long = pd.DataFrame(rows_list)
df_long = df_long.fillna("missing")
df_long = df_long.fillna("missing")
# convert all to string except test_loss
# removing rows where test_loss is missing
df_long = df_long[df_long["test_loss"] != "missing"]
# same for epoch
df_long = df_long[df_long["epoch"] != "missing"]
df_long = df_long.applymap(str)
df_long["test_loss"] = df_long["test_loss"].astype(float)
df_long["epoch"] = df_long["epoch"].astype(int)
df_long["train_loss"] = df_long["train_loss"].astype(float)

# drop 	Unnamed: 0
def convert_embedding(name):
    if "GaussianEmbedder" in name:
        return "gaussian"
    elif "PositionalEncoding" in name:
        return "positional"
    elif "RandomFeature" in name:
        return "fourier"
    else:
        return "none"
df_long["embedding"] = df_long["embedding_function"].apply(convert_embedding)

# remove Unnamed: 0
df_long = df_long.drop(columns=["Unnamed: 0", "job_id"])


/var/folders/fy/__8z8cpn6gs04465sq9g1nq80000gn/T/ipykernel_35860/3372726706.py:50: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [194]:
df_long["length_scale"].unique()

array(['missing', '0.0001', '19.9526231496888', '0.0084580982817513',
       '6.5793322465756825', '0.0256502090568004', '0.0027890387320175',
       '0.077787370486943', '0.7153942654376494', '2.169519891498863',
       '0.2358996370701593', '0.0009196791985117', '0.0003032621305919'],
      dtype=object)

In [195]:
# convert name to match the arithemtic results
dic_rename = {
    "test_loss": "val_loss_total",
    "train_loss": "loss_total",
    "embedding": "embedding_type",
    "bandwidth": "gaussian_embedding_sd",
    "length_scale": "fourier_length_scale",
}
df_long = df_long.rename(columns=dic_rename)

# add column task
df_long["task"] = "sort"

# restrict to epoch == 49
df_long = df_long[df_long["epoch"] == 49]

In [196]:
results = pd.concat([results, df_long])

# replace nan with missing
results = results.fillna("missing")

In [197]:
results["task"]

1        arithmetic
1        arithmetic
1        arithmetic
1        arithmetic
1        arithmetic
            ...    
13799          sort
13849          sort
13899          sort
13949          sort
13999          sort
Name: task, Length: 251, dtype: object

In [198]:
df_long.columns

Index(['n_epochs', 'n_samples', 'seq_length', 'batch_size', 'lr',
       'weight_decay', 'task', 'verbose', 'no_positional_encoding',
       'no_layer_norm', 'fixed_embeddings', 'classif', 'device', 'save',
       'iter', 'left_bound', 'right_bound', 'gaussian_embedding_sd',
       'parametrize_std', 'in_seq_length', 'out_seq_length', 'n_layer',
       'n_head', 'n_embd', 'attention_only', 'revert_embedding',
       'embedding_function', 'read_in_bias', 'only_one_emb',
       'fourier_length_scale', 'kernel', 'holes', 'epoch', 'val_loss_total',
       'loss_total', 'embedding_type'],
      dtype='object')

In [199]:
df_long[df_long["embedding_type"] == "none"]

,n_epochs,n_samples,seq_length,batch_size,lr,weight_decay,task,verbose,no_positional_encoding,no_layer_norm,...,embedding_function,read_in_bias,only_one_emb,fourier_length_scale,kernel,holes,epoch,val_loss_total,loss_total,embedding_type
99,1200,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.108706,0.122374,none
2949,400,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.066832,0.051317,none
3349,400,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.034450,0.037317,none
3749,1200,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.272106,0.277998,none
4949,400,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.038771,0.023043,none
5349,400,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.038751,0.034214,none
5749,1200,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.186700,0.152284,none
6949,400,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.039867,0.042856,none
7349,1200,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.097850,0.082920,none
8549,1200,10000,10,128,0.0001,0,sort,0,False,False,...,missing,False,True,missing,missing,missing,49,0.215504,0.200082,none


In [200]:
results["epoch"]

1         1
1         1
1         1
1         1
1         1
         ..
13799    49
13849    49
13899    49
13949    49
13999    49
Name: epoch, Length: 251, dtype: int64

In [225]:
import numpy as np
import plotly.graph_objects as go

results_to_plot = results.copy()
#results_to_plot["dataset_path"] = results_to_plot["dataset_path"].str.split("/").str[2].str.split("tokenized_").str[1]

# filter to remove nans
#results_to_plot = results_to_plot.dropna(subset=["dataset_path"])

results_to_plot["epoch"] = results_to_plot["epoch"].astype(str)
#results_to_plot["num_layers"] = results_to_plot["num_layers"].astype(float)
filter_dic = {
    #"num_layers": "6",
    #"embedding_type": "gaussian",
    #"reverse_numerical_embedding": "False",
    #"d_model": "384",
    #"normalize_token_emb": "True",
    #"gaussian_embedding_sd": "[0.026041666666666668]",
    #"dataset_path": "fad_4_op_uniform",
    #"dataset_path": "fad_big",
    #"epoch": "49",
}
# allow missing or value in filter_dic
for key, value in filter_dic.items():
    results_to_plot = results_to_plot[np.isin(results_to_plot[key], [value, "missing"])]

# to_plot["emb_type_lengthscale"] = np.where(
#     to_plot["embedding_type"] == "linear", 
#     to_plot["embedding_type"],
#     np.where(
#         to_plot["embedding_type"] == "fourier", 
#         to_plot["embedding_type"] + " / " + to_plot["fourier_length_scale"].astype(str),
#         np.where(
#             to_plot["embedding_type"] == "gaussian", 
#             to_plot["embedding_type"] + " / " + to_plot["gaussian_embedding_sd"].astype(str),
#             to_plot["embedding_type"]  # Default case if other embedding types need handling
#         )
#     )
# )
results_to_plot = results_to_plot[results_to_plot["embedding_type"] != "none"]
results_to_plot["lengthscale"] = np.where(
    results_to_plot["embedding_type"] == "fourier",
    results_to_plot["fourier_length_scale"].astype(str),
    np.where(
        results_to_plot["embedding_type"] == "gaussian",
        results_to_plot["gaussian_embedding_sd"].astype(str).str.replace("[", "").str.replace("]", ""),
        "missing"
    )
)

print(len(results_to_plot))
# convert to float
results_to_plot["lengthscale"] = results_to_plot["lengthscale"].astype(float)
# filter to keep only lengthscale < 11
results_to_plot = results_to_plot[results_to_plot["lengthscale"] < 11]
results_to_plot = results_to_plot[results_to_plot["lengthscale"] != 1]

print(len(results_to_plot))




group_cols = list(results_to_plot.columns.difference(["loss_total", "val_loss_total", "iter", "collator", "val_loss_mlm", "val_loss_num", "val_loss_total", "loss_mlm", "loss_num"]))
results_to_plot = results_to_plot.groupby(group_cols).agg(
    loss_total=("loss_total", "mean"),
    val_loss_total=("val_loss_total", "mean"),
    # min and max
    loss_total_min=("loss_total", "min"),
    loss_total_max=("loss_total", "max"),
    val_loss_total_min=("val_loss_total", "min"),
    val_loss_total_max=("val_loss_total", "max"),

).reset_index()

print(len(results_to_plot))

# show remaining non-unique columns
for col in group_cols:
    if len(np.unique(results_to_plot[col])) > 1:
        # check if missing is in the unique values
        if "missing" in np.unique(results_to_plot[col]) and len(np.unique(results_to_plot[col])) == 2:
            continue
        elif len(np.unique(results_to_plot[col])) < 100:
            print(col, np.unique(results_to_plot[col], return_counts=True))
        else:
            print(col, len(np.unique(results_to_plot[col])))

# remove duplicates on group_cols
results_to_plot = results_to_plot.drop_duplicates(subset=group_cols)


# plot
import plotly.express as px

# random plot
fig = px.scatter(
    results_to_plot,
    x="lengthscale",
    y="val_loss_total",
    #facet_row="lengthscale",
    facet_col="embedding_type",
    facet_row="task",
    #color="dataset_path
    hover_data=group_cols,
    log_x=True,
    log_y=True,
    # don't share scale
)
fig.write_image("plots/random.pdf")


fig = px.scatter(
    results_to_plot,
    x="lengthscale",
    y="val_loss_total",
    #facet_row="lengthscale",
    facet_col="embedding_type",
    facet_row="task",
    #color="dataset_path
    hover_data=group_cols,
    log_x=True,
    log_y=True,
    # don't share scale
)

position_sort = 5 * 11 / 128
fig.add_shape(
    type="line",
    x0=position_sort, x1=position_sort,
    y0=0.01, y1=2,
    xref="x1", yref="paper",
    line=dict(color="black", width=2, dash="dot"),
    row=0, col=1  # Adjust these indices as necessary
)
fig.add_shape(
    type="line",
    x0=position_sort, x1=position_sort,
    y0=0.01, y1=2,
    xref="x1", yref="paper",
    line=dict(color="black", width=2, dash="dot"),
    row=0, col=0  # Adjust these indices as necessary
)

#x = 2 / 384
position_arithmetic = 5 * 2 / 384
fig.add_shape(
    type="line",
    x0=position_arithmetic, x1=position_arithmetic,
    y0=0.002, y1=1,
    xref="x2", yref="paper",
    line=dict(color="black", width=2, dash="dot"),
    row=1, col=0  # Adjust these indices as necessary
)

fig.add_shape(
    type="line",
    x0=position_arithmetic, x1=position_arithmetic,
    y0=0.002, y1=1,
    xref="x2", yref="paper",
    line=dict(color="black", width=2, dash="dot"),
    row=1, col=1  # Adjust these indices as necessary
)


# vertical line in x = 11 / 128 for task sort
# vertical line in x = 2 / 384 for task arithmetic

fig.update_yaxes(matches=None)
# bigger markers
fig.update_traces(marker=dict(size=10))

# change shape
fig.update_layout(
    #showlegend=False,
    #margin=dict(l=0, r=0, t=0, b=0),
    height=600,
    width=600,
    font=dict(
        size=18,
    )
)

# rename facet labels
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1].capitalize() + " Embedding"))

# rename axis
fig.update_xaxes(title_text="Lengthscale")
fig.update_yaxes(title_text="Validation Loss")


fig.show()
# save as pdf
fig.write_image("plots/random.pdf")

fig.write_image("plots/lengthscale_vs_val_loss.pdf")


# print model size

239
150
49
batch_size (array(['128', '256'], dtype=object), array([27, 22]))
embedding_function (array(['GaussianEmbedder()', 'RandomFeatureEmbedder()', 'missing'],
      dtype=object), array([11, 11, 27]))
embedding_type (array(['fourier', 'gaussian'], dtype=object), array([23, 26]))
epoch (array(['1', '49'], dtype=object), array([27, 22]))
fourier_length_scale (array(['0.0001', '0.0002848035868435', '0.0003032621305919',
       '0.0008111308307896', '0.0009196791985117', '0.0023101297000831',
       '0.0027890387320175', '0.0065793322465756', '0.0084580982817513',
       '0.0187381742286038', '0.0256502090568004', '0.053366992312063',
       '0.077787370486943', '0.1519911082952933', '0.2358996370701593',
       '0.4328761281083056', '0.7153942654376494', '1.0',
       '1.232846739442066', '10.0', '2.169519891498863',
       '3.511191734215127', '6.5793322465756825', 'missing'], dtype=object), array([ 2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       15,  1,  

In [184]:
results_to_plot[]

,attention_only,batch_size,classif,context_length,d_model,dataset,dataset_path,device,dim_feedforward,dropout,...,verbose,vocab_size,warmup_steps,weight_decay,loss_total,val_loss_total,loss_total_min,loss_total_max,val_loss_total_min,val_loss_total_max
0,True,256,False,missing,missing,missing,missing,cpu,missing,missing,...,0,missing,missing,0,2.581588,2.308185,2.070450,3.368551,2.191681,2.482187
1,True,256,False,missing,missing,missing,missing,cpu,missing,missing,...,0,missing,missing,0,2.630534,2.260017,2.022616,3.728161,2.206849,2.353458
2,True,256,False,missing,missing,missing,missing,cpu,missing,missing,...,0,missing,missing,0,2.108235,2.193932,1.956410,2.224594,2.160581,2.238713
3,True,256,False,missing,missing,missing,missing,cpu,missing,missing,...,0,missing,missing,0,1.890690,2.123059,1.442712,2.701913,1.991610,2.239720
4,True,256,False,missing,missing,missing,missing,cpu,missing,missing,...,0,missing,missing,0,0.992541,1.024153,0.758503,1.413196,0.790088,1.470421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,missing,128,missing,955,384,fad,./data/tokenized_fad_4_op_uniform/tokenized_ds...,missing,1536,0.1,...,missing,27,2000,0.1,0.590047,0.587384,0.590047,0.590047,0.587384,0.587384
128,missing,128,missing,955,384,fad,./data/tokenized_fad_4_op_uniform/tokenized_ds...,missing,1536,0.1,...,missing,27,2000,0.1,0.589994,0.587893,0.589994,0.589994,0.587893,0.587893
129,missing,128,missing,955,384,fad,./data/tokenized_fad_4_op_uniform/tokenized_ds...,missing,1536,0.1,...,missing,27,2000,0.1,0.590063,0.587451,0.590063,0.590063,0.587451,0.587451
130,missing,128,missing,955,384,fad,./data/tokenized_fad_4_op_uniform/tokenized_ds...,missing,1536,0.1,...,missing,27,2000,0.1,0.590252,0.588591,0.590252,0.590252,0.588591,0.588591


In [185]:
# show remaining non-unique columns
df_plot = results_to_plot[results_to_plot["task"] == "arithmetic"]
for col in group_cols:
    if len(np.unique(df_plot[col])) > 1:
        # check if missing is in the unique values
        if "missing" in np.unique(df_plot[col]) and len(np.unique(df_plot[col])) == 2:
            continue
        elif len(np.unique(df_plot[col])) < 100:
            print(col, np.unique(df_plot[col], return_counts=True))
        else:
            print(col, len(np.unique(df_plot[col])))

embedding_type (array(['fourier', 'gaussian'], dtype=object), array([62, 48]))
gaussian_embedding_sd (array(['0.0001', '0.0035938136638046', '0.1291549665014884',
       '10000000000.0', '166.8100537200059', '215443.46900318863',
       '278255940.2207126', '4.641588833612782', '5994.842503189409',
       '7742636.826811277', '[0.00028480358684358016]',
       '[0.0008111308307896872]', '[0.002310129700083158]',
       '[0.0035938136638046254]', '[0.006579332246575682]',
       '[0.01873817422860383]', '[0.05336699231206307]',
       '[0.1291549665014884]', '[0.1519911082952933]',
       '[0.43287612810830567]', '[1.232846739442066]', '[10.0]',
       '[3.511191734215127]', '[4.641588833612782]',
       '[9.999999999999999e-05]'], dtype=object), array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 6, 6, 3, 6, 5, 6, 3, 6, 6, 6, 6,
       5, 3, 9]))
lengthscale (array([1.00000000e-04, 2.84803587e-04, 8.11130831e-04, 2.31012970e-03,
       3.59381366e-03, 6.57933225e-03, 1.87381742e-02, 5.33669923e-02

In [ ]:
df_plot[]

In [ ]:
results

In [ ]:
df_plot["attention_only"] == "False"